#Pythonで学ぶ画像認識　第3章 深層学習を使う準備
## 第3.3節 PyTorchの基礎

###モジュールのインポート

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

###テンソルの生成

In [2]:
t1 = torch.tensor([1, 2, 3, 4])
t2 = torch.zeros((32, 3, 128, 128))

print(f't1 = {t1}, t1.shape = {t1.shape}')
print(f't2.shape = {t2.shape}')

t1 = tensor([1, 2, 3, 4]), t1.shape = torch.Size([4])
t2.shape = torch.Size([32, 3, 128, 128])


###テンソルのGPUへの転送

In [3]:
t1 = torch.tensor([1, 2, 3, 4], device='cuda')

t2 = torch.tensor([1, 2, 3, 4])
t2 = t2.to('cuda')

###Python演算子を使ったテンソルの演算

In [4]:
t1 = torch.tensor([1, 2, 3, 4])
t2 = torch.tensor([2, 4, 6, 8])

t3 = t1 + t2
t4 = t1 ** 2

print(f't3 = {t3}')
print(f't4 = {t4}')

t3 = tensor([ 3,  6,  9, 12])
t4 = tensor([ 1,  4,  9, 16])


###view関数によるテンソルの形の変更

In [5]:
t1 = torch.tensor([1, 2, 3, 4])
t1 = t1.view(2, 2)

t2 = torch.tensor([1, 2, 3, 4, 5, 6])
t2 = t2.view(2, -1)

print(f't1.shape = {t1.shape}')
print(f't2.shape = {t2.shape}')

t1.shape = torch.Size([2, 2])
t2.shape = torch.Size([2, 3])


###transpose関数とpermute関数による軸の並び替え

In [6]:
t1 = torch.zeros((32, 3, 128, 128))
t1 = t1.transpose(0, 2)

t2 = torch.zeros((32, 3, 128, 128))
t2 = t2.permute(2, 0, 3, 1)

print(f't1.shape = {t1.shape}')
print(f't2.shape = {t2.shape}')

t1.shape = torch.Size([128, 3, 32, 128])
t2.shape = torch.Size([128, 32, 128, 3])


###cat関数とstack関数による複数テンソルの連結

In [7]:
t1 = torch.tensor([1, 2, 3, 4, 5, 6]).view(2, 3)
t2 = torch.tensor([7, 8, 9]).view(1, 3)
t3 = torch.cat((t1, t2))

t4 = torch.tensor([1, 2, 3])
t5 = torch.tensor([4, 5, 6])
t6 = torch.stack((t4, t5), dim=1)

print(f't3 = {t3}')
print(f't6 = {t6}')

t3 = tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
t6 = tensor([[1, 4],
        [2, 5],
        [3, 6]])


###インデクシングによるテンソルの要素の抽出

In [8]:
t1 = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9]).view(3, 3)
t2 = t1[[0, 1]]
t3 = t1[:, [0, 2]]
t4 = t1[[0, 2, 1], [1, 2, 1]]
t5 = t1[[True, False, False]]
t6 = t1[t1 % 2 == 0]

print(f't2 = {t2}')
print(f't3 = {t3}')
print(f't4 = {t4}')
print(f't5 = {t5}')
print(f't6 = {t6}')

t2 = tensor([[1, 2, 3],
        [4, 5, 6]])
t3 = tensor([[1, 3],
        [4, 6],
        [7, 9]])
t4 = tensor([2, 9, 5])
t5 = tensor([[1, 2, 3]])
t6 = tensor([2, 4, 6, 8])


###ブロードキャストを使った演算

In [9]:
t1 = torch.tensor([1, 2]).view(2, 1)
t2 = torch.tensor([3, 4, 5])
# t1 = [[1],         ->      [[1, 1, 1],
#       [2]]     broadcast    [2, 2, 2]]
# t2 =  [3, 4, 5]    ->      [[3, 4, 5],
#                             [3, 4, 5]]
t3 = t1 + t2

t4 = torch.tensor([1, 2, 3, 4, 5, 6]).view(3, 2)
t5 = torch.tensor([3, 4])
# t4 = [[1, 2],
#       [3, 4],
#       [5, 6]]  broadcast    
# t5 =  [3, 4]       ->      [[3, 4],
#                             [3, 4],
#                             [3, 4]]
t6 = t4 + t5

print(f't3 = {t3}')
print(f't6 = {t6}')

t3 = tensor([[4, 5, 6],
        [5, 6, 7]])
t6 = tensor([[ 4,  6],
        [ 6,  8],
        [ 8, 10]])


###多クラスロジスティック回帰のPyTorchを使った実装

In [10]:
class MultiClassLogisticRegression(nn.Module):
    '''
    多クラスロジスティック回帰
    dim_input  : 入力次元
    num_classes: 分類対象の物体クラス数
    '''
    def __init__(self, dim_input: int, num_classes: int):
        super().__init__()
        
        self.linear = nn.Linear(dim_input, num_classes)

    '''
    順伝播関数
    x: 入力データ, [バッチサイズ, 入力次元]
    '''
    def forward(self, x: torch.Tensor):
        l = self.linear(x)
        y = l.softmax(dim=1)

        return y

###多クラスロジスティック回帰モデルの使用例

In [11]:
model = MultiClassLogisticRegression(32 * 32 * 3, 10)

# 学習モードに設定
model.train()

# 評価(推論)モードに設定
model.eval()

x = torch.normal(0, 1, size=(1, 32 * 32 * 3))
y = model(x)

for name, parameter in model.named_parameters():
    print(f'{name}: shape = {parameter.shape}')

linear.weight: shape = torch.Size([10, 3072])
linear.bias: shape = torch.Size([10])


###Sequentialクラスを使ったモデルの実装

In [12]:
class FNNSequential(nn.Module):
    '''
    順伝播型ニューラルネットワーク
    dim_input  : 入力次元
    num_classes: 分類対象の物体クラス数
    '''
    def __init__(self, dim_input: int, num_classes: int):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(dim_input, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    '''
    順伝播関数
    x: 入力データ, [バッチサイズ, 入力次元]
    '''
    def forward(self, x):
        l = self.layers(x)
        y = l.softmax(dim=1)

        return y


###ModuleListクラスを使ったモデルの実装

In [13]:
class FNNModuleList(nn.Module):
    '''
    順伝播型ニューラルネットワーク
    dim_input  : 入力次元
    num_classes: 分類対象の物体クラス数
    '''
    def __init__(self, dim_input: int, num_classes: int):
        super().__init__()
        
        layers = [nn.Linear(dim_input, 256)]
        layers += [nn.Linear(256, 256) for _ in range(2)]
        layers.append(nn.Linear(256, num_classes))
        self.layers = nn.ModuleList(layers)

    '''
    順伝播関数
    x: 入力データ, [バッチサイズ, 入力次元]
    '''
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        l = self.layers[-1](x)
        y = l.softmax(dim=1)
        
        return y

###FNNモデルの使用例

In [14]:
model_sequential = FNNSequential(32 * 32 * 3, 10)
model_modulelist = FNNModuleList(32 * 32 * 3, 10)

model_sequential.eval()
model_modulelist.eval()

x = torch.normal(0, 1, size=(1, 32 * 32 * 3))
y_sequential = model_sequential(x)
y_modulelist = model_modulelist(x)

print(f'y_sequential = {y_sequential}')
print(f'y_modulelist = {y_modulelist}')

y_sequential = tensor([[0.0960, 0.1047, 0.1014, 0.1010, 0.1016, 0.0950, 0.1016, 0.0966, 0.1024,
         0.0997]], grad_fn=<SoftmaxBackward0>)
y_modulelist = tensor([[0.0938, 0.1071, 0.1018, 0.0970, 0.0940, 0.0996, 0.1057, 0.1047, 0.0989,
         0.0973]], grad_fn=<SoftmaxBackward0>)


###自動微分を使った誤差逆伝播の例

In [16]:
linear = nn.Linear(32 * 32 * 3, 10)

# 入力とラベルの用意
x = torch.normal(0, 1, size=(1, 32 * 32 * 3))
y = torch.tensor([0])

# 目的関数(交差エントロピー誤差)の計算
y_pred = linear(x)
loss = F.cross_entropy(y_pred, y)

# 誤差逆伝播
loss.backward()

print(f'linear.weight.grad = {linear.weight.grad}')
print(f'linear.bias.grad = {linear.bias.grad}')

linear.weight.grad = tensor([[-1.0675, -0.0844, -0.2668,  ...,  1.3030,  0.7700, -0.1012],
        [ 0.1880,  0.0149,  0.0470,  ..., -0.2295, -0.1356,  0.0178],
        [ 0.1322,  0.0105,  0.0330,  ..., -0.1613, -0.0953,  0.0125],
        ...,
        [ 0.1230,  0.0097,  0.0307,  ..., -0.1501, -0.0887,  0.0117],
        [ 0.1460,  0.0115,  0.0365,  ..., -0.1782, -0.1053,  0.0138],
        [ 0.1647,  0.0130,  0.0412,  ..., -0.2010, -0.1188,  0.0156]])
linear.bias.grad = tensor([-0.8975,  0.1581,  0.1111,  0.0302,  0.0590,  0.0767,  0.0978,  0.1034,
         0.1228,  0.1385])
